In [1]:
import pandas as pd
import numpy as np

In [3]:
def fpre_process(fname):
    # Hot encoding for all categorical columns
    # Impute values for nulls in categorical and numerical columns

    # 1. Get the selected labels and features - Done
    # 2. Determine the cat / numerical in features - Done
    # 3. Determine columns with Nulls - Done
    # 4. Determine most frequent in Cat - Done
    # 5. Impute cat column values with most frequent - Done
    # 6. Detemine mean / average for Num - Done
    # 7. Impute num column values with mean - Done
    # 8. Hot Encode - Done
    # 9. Standardize - TBD
    # 10. Return X and y - Done
    # 11. Include label in the pre-process

    # Get the labels and features selected

    from label_features import flabel_features
    olabel, ofeatures = flabel_features() # uncomment when testing is done


    def load_process():
        from load import fload_csv
        df, size = fload_csv(fname)
    # Store in temp dataframe
        tdf = df[ofeatures].copy()
        ldf = df[olabel].copy()
        return tdf, ldf

    def null_process(tdf, ldf):

    # calculate the numerical and cat columns

        cols = list(tdf.columns)
        num_cols = list(tdf._get_numeric_data().columns)
        cat_cols = list(set(cols) - set(num_cols))

    # Determine columns with Null values and Replace NAs

        null_num_cols = tdf[num_cols].columns[tdf[num_cols].isnull().any()
                                          ].tolist()
        null_cat_cols = tdf[cat_cols].columns[tdf[cat_cols].isnull().any()
                                          ].tolist()

    # For numerical columns

        for col in null_num_cols:
            tdf[col].fillna(tdf[col].mean(), inplace=True)

    # For categorical columns impute and hot encode

        for col in null_cat_cols:
        # impute missing values
            tdf[col].fillna(tdf[col].value_counts().max(), inplace=True)

        for col in cat_cols:
            # Hot One Encode
            tdf = pd.concat([tdf, pd.get_dummies(tdf[col])], axis=1)

        tdf.drop(cat_cols, axis=1, inplace=True)
        return tdf, ldf


    tdf, ldf = load_process()
    tdf, ldf = null_process(tdf, ldf)

    # Extract X and y
    y = ldf.as_matrix().astype(np.float)
    X = tdf.as_matrix().astype(np.float)

    # reshape for cross validation - sheez!
    c, r = y.shape
    y = y.reshape(c,)

    return X, y

In [4]:
X, y = fpre_process('train.csv')


 Data successfully loaded


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn import metrics
def ds(X, y):
    cm = DecisionTreeClassifier()
    cv = StratifiedKFold(n_splits=20)
    ypred = cross_val_predict(cm, X,y, cv=cv, n_jobs=-1)
    yscore = cross_val_predict(cm, X,y, cv=cv, n_jobs=-1, method='predict_proba')
    score = metrics.accuracy_score(y, ypred)
    return score, ypred, yscore

In [29]:
score, ypred, yscore = ds(X,y)
print score

0.81593714927


In [78]:
# Confusion Matrix
cmatrix = metrics.confusion_matrix(y, ypred)
print type(cmatrix)
TP = cmatrix[0,0]
TN = cmatrix[1,1]
FP = cmatrix[0,1]
FN = cmatrix[1,0]

Specificity = ((TN*1.0)/(TN+FP))
Sensitivity = ((TP*1.0)/(TP+FN))
Accuracy = ((TP+TN)*1.0)/(TN+TP+FP+FN)
Error_Rate = ((FP+FN)*1.0)/(TN+TP+FP+FN)
Precision = ((TP*1.0)/(TP+FP))
FPR = ((FP*1.0)/(TN+FP))
f1score = (2*Precision*Sensitivity) / (Precision + Sensitivity)
skf1score = metrics.f1_score(y, ypred, average='weighted')
neglogloss = metrics.log_loss(y, ypred, eps=1e-15, normalize=True)
mcc = metrics.matthews_corrcoef(y, ypred)

print '{:03.2f}'.format(Specificity)
print Sensitivity, Accuracy, Error_Rate, Precision, FPR, f1score, skf1score, neglogloss, mcc

# Confusion Matrix Related Metrics


Avg_Precision_Score = metrics.average_precision_score(y, yscore[:,1], average='macro')

# ROC Curve

fpr, tpr, thresholds = metrics.roc_curve(y, yscore[:,1])
auc_score = metrics.roc_auc_score(y, yscore[:,1])
print auc_score
# Precision Curve

precision, recall, thresholds = metrics.precision_recall_curve(y, yscore[:,1])



<type 'numpy.ndarray'>
0.78
0.833622183709 0.81593714927 0.18406285073 0.876138433515 0.216560509554 0.854351687389 0.814297504351 6.35736666823 0.606152748139
0.797718339565


In [ ]:
# Dict Test
def f(i):
    return {
        'decision_tree': decision_tree,
        'knn': knn
    }.get(i, decision_tree)

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

# Import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binarize the output
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                    random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=random_state))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [61]:
score = accuracy_score(ytest, ypred)

In [62]:
score

0.84090909090909094

In [ ]:
# Feature Extraction with RFE
from pandas import read_csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# load data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(url, names=names)
print dataframe.head(10)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
# feature extraction
model = LogisticRegression()
rfe = RFE(model, 3)
fit = rfe.fit(X, Y)
print("Num Features: %d") % fit.n_features_
print("Selected Features: %s") % fit.support_
print("Feature Ranking: %s") % fit.ranking_

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [285]:
l = [[1,2,3],[4,5,6],[7,8,9]]
f1 = [1, 2, 3]
t1 = [3, 4, 5]
f2 = [6, 7, 8]
t2 = [10, 11, 12]
d = []

In [ ]:
for i in range(4):
    

In [286]:
st1 = ['m1' for i in f1]
st2 = ['m2' for i in f2]
d.append(st1)
d.append(f1)
d.append(t1)
dn = np.array(d).T

In [287]:
dn

array([['m1', '1', '3'],
       ['m1', '2', '4'],
       ['m1', '3', '5']], 
      dtype='|S2')

In [288]:
d = []
d.append(st2)
d.append(f2)
d.append(t2)
dn1 = np.array(d).T

In [289]:
dn1

array([['m2', '6', '10'],
       ['m2', '7', '11'],
       ['m2', '8', '12']], 
      dtype='|S2')

In [292]:

dn = np.append(dn, dn1, axis =0)

In [293]:
dn

array([['m1', '1', '3'],
       ['m1', '2', '4'],
       ['m1', '3', '5'],
       ['m2', '6', '10'],
       ['m2', '7', '11'],
       ['m2', '8', '12']], 
      dtype='|S2')

In [273]:
df=pd.DataFrame(dnx,columns=['model_name','FPR', 'TPR'])

In [297]:
gh = np.zeros(shape=(3,3))

In [274]:
df

,model_name,FPR,TPR
